# Downloading and processing the data for one district

https://www.elastic.co/guide/en/elasticsearch/reference/current/docs-update-by-query.html to update and clean the data ?

In [50]:
zipcode = 75008

### Imports & connection to Elasticsearch

In [51]:
from elasticsearch import Elasticsearch
from ssl import create_default_context
import pandas as pd
import numpy as np
import sys
import datetime

with open('connect.txt') as f:
    str = f.readlines()

es = Elasticsearch(str)

### Change maximum size setting
By default the query returns the first 10,000 hits

In [52]:
es.indices.put_settings(
    index="carshare_car_history",
    body={
        "index.max_result_window": 500000
    }

)

{'acknowledged': True}

## Query
Fetch all data in given district. This should be repeated for each zipcode.

In [53]:
result = es.search(
  index="carshare_car_history",
  body = {
  "size": 200000,
  "query": {

      "bool": {
        "must": {
          "match_all": {}
        },
        "filter": [
    {
    "geo_shape": {
      "ignore_unmapped": "true",
      "location": {
        "relation": "INTERSECTS",
        "shape": {
          "coordinates": [
            [
              [
                1.87411, 49.08001
              ],
              [
                1.87411,
                48.64617
              ],
              [
                2.72473,
                48.64617
              ],
              [
                2.72473,
                49.08001
              ],
              [
                1.87411,
                49.08001
              ]
            ]
          ],
          "type": "Polygon"
        }
      }
    }
  },
  
  {
      "match_phrase": {
        "brand": "Zity"
      }    
  },
  
{
      "match_phrase": {
        "zipcode": zipcode
      }    
  },

  {
      "range": { 
        "last_update": {
          "gte": "2020-01-01T00:00:00",
          "lte": "2022-05-09T00:00:00" # fetch data until May 10th, 2022
          }
        }}
        ]
  
    }

  }
},
request_timeout=30 # default timeout is 10sec
 
)

In [54]:
print(len(result['hits']['hits']))

31745


## Convert data to dataframe

In [71]:
df = pd.json_normalize(result['hits']['hits'])
df = df[['_source.end__date', '_source.car_plate_number', '_source.status', '_source.group_id', '_source.duration', '_source.distance', '_source.location', '_source.zipcode', '_source.battery', '_source.end_battery', '_source.start_date']]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31745 entries, 0 to 31744
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   _source.end__date         31744 non-null  object 
 1   _source.car_plate_number  31745 non-null  object 
 2   _source.status            31745 non-null  object 
 3   _source.group_id          31745 non-null  object 
 4   _source.duration          31744 non-null  float64
 5   _source.distance          31744 non-null  float64
 6   _source.location          31745 non-null  object 
 7   _source.zipcode           31745 non-null  object 
 8   _source.battery           31741 non-null  float64
 9   _source.end_battery       31736 non-null  float64
 10  _source.start_date        31745 non-null  object 
dtypes: float64(4), object(7)
memory usage: 2.7+ MB


In [72]:
print(df['_source.group_id'].value_counts())
print(df['_source.status'].unique())
print((df['_source.end__date'].str[:7].value_counts()))
print(len(df['_source.car_plate_number'].unique()))

Zity              25946
Client             2564
Maintenance        1680
Workshop            954
Battery             563
Zity Corporate       20
Airport               8
14                    6
MAINTENANCE           4
Name: _source.group_id, dtype: int64
['BOOKED' 'FREE' 'BOOKED_PARKED' 'RESERVED' 'UNRESPONSIVE']
2021-08    2740
2021-06    2626
2021-09    2599
2021-07    2585
2021-11    2526
2021-12    2464
2021-10    2267
2021-05    2202
2021-02    2035
2021-03    1793
2021-04    1792
2020-12    1587
2021-01    1506
2022-01    1071
2022-02     917
2022-03     352
2022-05     248
2020-11     228
1752-12     194
1753-01      10
0001-01       1
2022-04       1
Name: _source.end__date, dtype: int64
750


### Utils

In [73]:
month_duration_dict = {"Jan": 31, "Feb": 28, "Mar": 31, "Apr": 30, "May": 31, "Jun": 30, "Jul": 31, "Aug": 31, "Sep": 30, "Oct": 31, "Nov": 30, "Dec": 31}

durations = np.array(list(month_duration_dict.values()))
cum_durations = np.cumsum(durations)
cum_dict = {i+1: cum_durations[i] for i in range(len(cum_durations))}
cum_dict[0]=0

week_dict = {"Monday": 0, "Tuesday": 1, "Wednesday": 2, "Thursday": 3, "Friday": 4, "Saturday": 5, "Sunday": 6}

monthdict = {"Jan": 0, "Feb": 1, "Mar": 2, "Apr": 3, "May": 4, "Jun": 5, "Jul": 6, "Aug": 7, "Sep": 8, "Oct": 9, "Nov": 10,  "Dec": 11}

inv_month = {v: k for k, v in monthdict.items()}

### Map values

In [74]:
df['_source.group_id'].replace("Zity", "Client", inplace=True) # Map Zity to Client
df['_source.group_id'].replace("Zity Corporate", "Defleeted", inplace=True) # Map Zity Corporate to Defleeted
df.rename(columns = {'_source.end__date':'end_date', '_source.car_plate_number':'car_plate_number', '_source.status':'status', '_source.group_id':'group_id', '_source.duration':'kibana_duration', '_source.distance':'distance', '_source.location':'location', '_source.zipcode':'zipcode', '_source.start_date':'start_date'}, inplace = True)

### Splitting and creating columns

In [75]:
df['delta_battery']=df['_source.end_battery']-df['_source.battery']
df.drop('_source.end_battery', axis=1, inplace=True)
df.drop('_source.battery', axis=1, inplace=True)
df[['latitude', 'longitude']] = df['location'].str.split(',', expand=True)
df.drop('location', axis=1, inplace=True)
df['kibana_duration'] = pd.to_numeric(df['kibana_duration'], errors='coerce')
df['kibana_duration'] = df['kibana_duration'].replace(np.nan, 0)
df["kibana_duration"] = df["kibana_duration"].astype(int)
#df['kibana_duration'] = df['kibana_duration'].astype(str)
#df[['kibana_duration', 'trash']] = df['kibana_duration'].str.split('.', expand=True)
#df.drop('trash', axis=1, inplace=True)
df[['end_date', 'end_time']] = df['end_date'].str.split('T', expand=True)
df[['end_time', 'trash']] = df['end_time'].str.split('.', expand=True)
df.drop('trash', axis=1, inplace=True)
df[['end_year', 'end_month', "end_day_number"]] = df['end_date'].str.split('-', expand=True)
df.drop('end_date', axis=1, inplace=True)
df[['end_hour', 'end_minutes', "end_seconds"]] = df['end_time'].str.split(':', expand=True)
df.drop('end_time', axis=1, inplace=True)
df.drop('end_seconds', axis=1, inplace=True)
df[['start_date', 'start_time']] = df['start_date'].str.split('T', expand=True)
df[['start_time', 'trash']] = df['start_time'].str.split('.', expand=True)
df.drop('trash', axis=1, inplace=True)
df[['start_year', 'start_month', "start_day_number"]] = df['start_date'].str.split('-', expand=True)
df.drop('start_date', axis=1, inplace=True)
df[['start_hour', 'start_minutes', "start_seconds"]] = df['start_time'].str.split(':', expand=True)
df.drop('start_time', axis=1, inplace=True)
df.drop('start_seconds', axis=1, inplace=True)
df.head()

,car_plate_number,status,group_id,kibana_duration,distance,zipcode,delta_battery,latitude,longitude,end_year,end_month,end_day_number,end_hour,end_minutes,start_year,start_month,start_day_number,start_hour,start_minutes
0,FB-062-TV,BOOKED,Client,384,10.0,75008,-5.0,48.87389374,2.29586697,2022,02,18,18,54,2022,02,18,12,47
1,GB-907-JX,FREE,Client,490,21.0,75008,-11.0,48.87421417,2.31447959,2022,02,18,18,53,2022,02,18,11,01
2,FA-653-MN,BOOKED,Client,385,14.0,75008,-9.0,48.87096024,2.31553102,2022,02,18,18,54,2022,02,18,12,47
3,FZ-618-RF,FREE,Client,386,13.0,75008,-11.0,48.87348938,2.32222128,2022,02,18,18,53,2022,02,18,12,47
4,GB-857-JX,BOOKED,Client,114,16.0,75008,-8.0,48.86917114,2.31440592,2022,02,18,18,53,2022,02,18,17,21


### Change types

In [76]:
df['distance']= pd.to_numeric(df['distance'], errors='coerce')
df['distance'] = df['distance'].fillna(0)
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')
df['zipcode'] = pd.to_numeric(df['zipcode'], errors='coerce')
df['kibana_duration'] = pd.to_numeric(df['kibana_duration'], errors='coerce')
df['end_year'] = pd.to_numeric(df['end_year'], errors='coerce')
df['end_month'] = pd.to_numeric(df['end_month'], errors='coerce')
df['end_day_number'] = pd.to_numeric(df['end_day_number'], errors='coerce')
df['end_hour'] = pd.to_numeric(df['end_hour'], errors='coerce')
df['end_minutes'] = pd.to_numeric(df['end_minutes'], errors='coerce')
df['start_year'] = pd.to_numeric(df['start_year'], errors='coerce')
df['start_month'] = pd.to_numeric(df['start_month'], errors='coerce')
df['start_day_number'] = pd.to_numeric(df['start_day_number'], errors='coerce')
df['start_hour'] = pd.to_numeric(df['start_hour'], errors='coerce')
df['start_minutes'] = pd.to_numeric(df['start_minutes'], errors='coerce')

### Calculate end time since 2020

In [77]:
df['time_since_2020'] = (df['end_year']-2020)*365*24*60 + ((df['end_month']-1).map(cum_dict)+df['end_day_number'])*24*60 + df['end_hour']*60 + df['end_minutes']

### Sort by end date

In [78]:
df = df.sort_values(by='time_since_2020', ascending = False)
df.reset_index(drop=True, inplace = True)

## Calculation of the durations

Only the end dates are reliable. The start date for a given segment corresponds to the last end date for the same plate.

In [79]:
def latest_segment(index):
    # returns index of the latest segment for the same plate
    
    plate_segments = list(df.loc[df['car_plate_number']==df.loc[index, 'car_plate_number']].index)
    previous_segments = plate_segments[plate_segments.index(index)+1:]
    if not len(previous_segments):
        return -1
    return previous_segments[0]

In [80]:
def next_segment(index):
    plate_segments = list(df.loc[df['car_plate_number']==df.loc[index, 'car_plate_number']].index)
    if plate_segments.index(index)==0:
        return -1
    return plate_segments[plate_segments.index(index)-1]

In [81]:
def duration(index):
    # returns the duration for a given index, only takes into account days and time
    
    previous_index = latest_segment(index)
    if previous_index==-1:
        return 0
    return df.loc[index, 'time_since_2020'] - df.loc[previous_index, 'time_since_2020']

Durations should be calculated on data not restricted to a specific district otherwise it makes no sense to look for the last occurence of the same plate as the car have travelled tto another district !

In [82]:
#df['duration']=df.index.map(duration)
df['duration']=df['kibana_duration'] 
df

,car_plate_number,status,group_id,kibana_duration,distance,zipcode,delta_battery,latitude,longitude,end_year,...,end_day_number,end_hour,end_minutes,start_year,start_month,start_day_number,start_hour,start_minutes,time_since_2020,duration
0,GD-112-TR,BOOKED,Client,43,0.0,75008,-1.0,48.840244,2.256432,2022.0,...,9.0,0.0,44.0,2022,5,9,0,1,1.237004e+06,43
1,GD-112-TR,BOOKED_PARKED,Client,37,12.0,75008,-8.0,48.872852,2.310610,2022.0,...,9.0,0.0,1.0,2022,5,8,23,24,1.236961e+06,37
2,FA-272-MP,BOOKED,Client,41,3.0,75008,-1.0,48.875664,2.302727,2022.0,...,8.0,23.0,22.0,2022,5,8,22,41,1.236922e+06,41
3,FA-272-MP,FREE,Client,120,0.0,75008,-1.0,48.875664,2.302727,2022.0,...,8.0,22.0,41.0,2022,5,8,20,42,1.236881e+06,120
4,FB-456-TV,BOOKED,Client,42,24.0,75008,-9.0,48.906239,2.358597,2022.0,...,8.0,20.0,43.0,2022,5,8,20,1,1.236763e+06,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31740,GB-992-JX,BOOKED_PARKED,Client,0,0.0,75008,-84.0,48.879951,2.318019,1752.0,...,31.0,23.0,50.0,2021,12,30,13,24,-1.403338e+08,0
31741,GB-805-JX,BOOKED,Client,0,0.0,75008,-51.0,48.872900,2.314600,1752.0,...,31.0,23.0,50.0,2022,1,3,11,0,-1.403338e+08,0
31742,FB-288-JR,FREE,Client,0,0.0,75008,-70.0,48.878600,2.312400,1752.0,...,31.0,23.0,50.0,2021,7,1,12,59,-1.403338e+08,0
31743,GB-097-JY,BOOKED,Client,0,0.0,75008,-4.0,48.870020,2.306030,1.0,...,1.0,0.0,0.0,2022,1,21,15,16,-1.061185e+09,0


## Next segment group-id column

In [83]:
def next_group_id(index):
    next_index = next_segment(index)
    if next_index == -1:
        return "no next plate"
    else :
        return df.loc[next_index, 'group_id']
df['next_group_id']=df.index.map(next_group_id)

## Keep only "CLIENT" Group-ids

In [84]:
df = df[df['group_id']=="Client"]

## Correct bugs

### Fake booking BUG

Some segments have a "Booked" status but no distance is covered. The status of these fake entries is changed to "FREE"

Unit test : index #255 & #256 (GB-102-JY)

In [85]:
def new_status(index):
    status = df.loc[index, 'status']
    distance = df.loc[index, 'distance']
    if (status=="BOOKED" or status=="BOOKED_PARK") and distance==0:
        return "FREE"
    return status

df['status']=df.index.map(new_status)

<ipython-input-85-7dd7bf9ab8d1>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['status']=df.index.map(new_status)


### Missing locations BUG

In April, 2022, the locations are missins. The corresponding rows should be deleted

In [86]:
df = df[df['latitude'].notna()]
df = df[df['longitude'].notna()]

### Fake year BUG
Some data are from 1753

In [87]:
df = df[df['end_year']>2000]

## Interpolation of useless segments

For a given plate, a FREE segment should be followed in time by a BOOKED or BOOKED_PARK segment and vice-versa. 

Unit test : index #253 -> #283 (GB-102-JY) #628 (GB-029-JY)

TODO : CHANGE START TIMES...

In [88]:
def has_changed_status(index):
    last_index = latest_segment(index)
    if last_index==-1:
        return True
    return df.loc[index, 'status']!=df.loc[last_index, 'status']

In [89]:
df['Status_has_changed']=df.index.map(has_changed_status)

In [90]:
def new_feature(index, feature_name):
    # returns the feature value for the last segment which status didn't change
    
    if df.loc[index, 'Status_has_changed']:
        return df.loc[index, feature_name]
    else :
        index_list = list(df.loc[df['car_plate_number']==df.loc[index, 'car_plate_number']].index)
        bool_list = list(df.loc[df['car_plate_number']==df.loc[index, 'car_plate_number']]['Status_has_changed'])
        ind = index_list.index(index)
        while not bool_list[ind] and ind < len(index_list):
            ind+=1 # Looking for the last segment for which the status has changed
        return df.loc[index_list[ind], feature_name]
    
def new_feature_cum(index, feature_name):
    # returns the sum of all values of the feature on the segments to interpolate
    if df.loc[index, 'Status_has_changed']:
        return df.loc[index, feature_name]
    else :
        value = int(df.loc[index, feature_name])
        index_list = list(df.loc[df['car_plate_number']==df.loc[index, 'car_plate_number']].index)
        bool_list = list(df.loc[df['car_plate_number']==df.loc[index, 'car_plate_number']]['Status_has_changed'])
        ind = index_list.index(index)
        while not bool_list[ind] and ind < len(index_list):
            ind+=1
            value += int(df.loc[index_list[ind], feature_name])
        return value

In [91]:
features_to_change = ['latitude', 'longitude', "start_year", "start_month", "start_day_number", "start_hour", "start_minutes"]
features_to_cumulate = ['distance', 'delta_battery']

for name in features_to_change :
    df['new_'+name] = df.index.map(lambda x: new_feature(x, name))
    

for name in features_to_cumulate :
    df['new_'+name] = df.index.map(lambda x: new_feature_cum(x, name))

In [92]:
for name in features_to_change :
    df[name] = df['new_'+name]
    df.drop('new_'+name, axis=1, inplace=True)

for name in features_to_cumulate :
    df[name] = df['new_'+name]
    df.drop('new_'+name, axis=1, inplace=True)

In [93]:
# Delete lines
def to_keep(index):
    next_index = next_segment(index)
    if next_index == -1 or df.loc[next_index, 'Status_has_changed']:
        return True
    return False

In [94]:
df["to_keep"]=df.index.map(to_keep)
df
df = df[df["to_keep"]]
df.drop("to_keep", axis=1, inplace=True)
df.drop("Status_has_changed", axis=1, inplace=True)
# UNIT TEST : GB-102-JY (done!)

## Start date column

$startdate = endate - duration$

In [95]:
def start_date(index):
    time = df.loc[index, 'time_since_2020'] - df.loc[index, 'kibana_duration'] # Start time since 2020
    year = 2020 + time // (365*24*60)
    time = time % (365*24*60)
    i = 0
    while cum_durations[i]< (time // (24*60)) and i < 12:
        i+=1
    month = inv_month[i]
    day = time // (24*60) - cum_durations[i-1] if i else time // (24*60)
    time = time % (24*60)
    hour = time // 60
    minute = time % 60
    return year, month, day, hour, minute
  
    
df['year']=df.index.map(lambda x: start_date(x)[0])
df['month']=df.index.map(lambda x: start_date(x)[1])
df['day_number']=df.index.map(lambda x: start_date(x)[2])
df['hour']=df.index.map(lambda x: start_date(x)[3])
df['minute']=df.index.map(lambda x: start_date(x)[4])

In [96]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24958 entries, 0 to 31538
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   car_plate_number  24958 non-null  object 
 1   status            24958 non-null  object 
 2   group_id          24958 non-null  object 
 3   kibana_duration   24958 non-null  int64  
 4   distance          24958 non-null  float64
 5   zipcode           24958 non-null  int64  
 6   delta_battery     24957 non-null  float64
 7   latitude          24958 non-null  float64
 8   longitude         24958 non-null  float64
 9   end_year          24958 non-null  float64
 10  end_month         24958 non-null  float64
 11  end_day_number    24958 non-null  float64
 12  end_hour          24958 non-null  float64
 13  end_minutes       24958 non-null  float64
 14  start_year        24958 non-null  int64  
 15  start_month       24958 non-null  int64  
 16  start_day_number  24958 non-null  int64 

In [97]:
df['end_month'].describe()

count    24958.000000
mean         6.989983
std          3.476269
min          1.000000
25%          4.000000
50%          7.000000
75%         10.000000
max         12.000000
Name: end_month, dtype: float64

### Day of the week
scripted_day_of_week contains many nans, so we recalculate the day of week with the datetime module. Actually __scripted day of week__ is mostly wrong as there are 23,191 entries with __scripted day of week__ $\neq$ __day of week__.

In [100]:
def get_week_day(index):
    day = int(df.loc[index, 'day_number'])
    month = int(monthdict[df.loc[index, 'month']]+1)
    year = int(df.loc[index, 'year'])
    if day==0:
        # Bug: 179 entries with date 2021-01-0, scripted_day_of_week was indicating 3 in isoweekday
        return 2
    return datetime.date(year, month, day).weekday()

df['day_of_week']=df.index.map(get_week_day)

In [101]:
print(np.count_nonzero(df['hour']-df['start_hour']))
print((df['hour']-df['start_hour']).describe())

2899
count    24958.000000
mean        -0.037463
std          2.218139
min        -23.000000
25%          0.000000
50%          0.000000
75%          0.000000
max         23.000000
dtype: float64


In [102]:
df.describe()

,kibana_duration,distance,zipcode,delta_battery,latitude,longitude,end_year,end_month,end_day_number,end_hour,...,start_day_number,start_hour,start_minutes,time_since_2020,duration,year,day_number,hour,minute,day_of_week
count,24958.000000,2.495800e+04,24958.0,24957.000000,24958.000000,24958.000000,24958.000000,24958.000000,24958.000000,24958.000000,...,24958.000000,24958.000000,24958.000000,2.495800e+04,24958.000000,24958.000000,24958.000000,24958.000000,24958.000000,24958.000000
mean,155.305113,-6.936905e+03,75008.0,-1.692217,48.874270,2.313007,2021.024000,6.989983,15.775182,14.090352,...,15.827470,13.849667,29.177057,8.230723e+05,155.305113,2021.025723,15.706467,13.812205,29.298501,3.215282
std,1582.557831,7.845759e+05,0.0,6.181302,0.005219,0.009891,0.352929,3.476269,8.916300,5.253418,...,8.929829,5.141637,17.352806,1.713593e+05,1582.557831,0.350243,8.919571,5.134294,17.363375,2.000544
min,0.000000,-1.023300e+07,75008.0,-322.000000,48.796696,2.214271,2020.000000,1.000000,1.000000,0.000000,...,1.000000,0.000000,0.000000,4.733670e+05,0.000000,2020.000000,0.000000,0.000000,0.000000,0.000000
25%,10.000000,0.000000e+00,75008.0,-2.000000,48.870600,2.303600,2021.000000,4.000000,8.000000,11.000000,...,8.000000,10.000000,14.000000,6.950618e+05,10.000000,2021.000000,8.000000,10.000000,14.000000,2.000000
50%,24.000000,0.000000e+00,75008.0,0.000000,48.874214,2.313900,2021.000000,7.000000,16.000000,15.000000,...,16.000000,15.000000,29.000000,8.304925e+05,24.000000,2021.000000,16.000000,15.000000,29.000000,3.000000
75%,82.000000,4.000000e+00,75008.0,0.000000,48.878600,2.322100,2021.000000,10.000000,24.000000,18.000000,...,24.000000,17.000000,44.000000,9.574015e+05,82.000000,2021.000000,24.000000,17.000000,44.000000,5.000000
max,155209.000000,1.802682e+07,75008.0,150.000000,48.962547,2.483372,2022.000000,12.000000,31.000000,23.000000,...,31.000000,23.000000,59.000000,1.237004e+06,155209.000000,2022.000000,31.000000,23.000000,59.000000,6.000000


In [104]:
# remove wrong dates (2021/01/00),
df = df[df['day_number']!=0]
df = df[df['end_day_number']!=0]
# drop duplicate columns
# df.drop("duration", axis=1, inplace=True)

# Change types
df['day_number'] = df['day_number'].astype(int)
df['year'] = df['year'].astype(int)

## Export csv

In [105]:
df.to_csv(f"{zipcode}.csv")